In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lightning
import torch
from torch.utils.data import random_split
import matplotlib.pyplot as plt

from point_clouds import viz
from point_clouds.datasets import FurnitureDataset
from point_clouds.models import PointCloudsModel, PointCloudsModelHParams

In [3]:
torch.autograd.set_grad_enabled(False)

In [4]:
lightning.seed_everything(42)

Global seed set to 42


42

In [5]:
plt.rc("figure", dpi=200)
plt.rc("legend", fontsize=6)

In [6]:
hparams = dict(
    inputs=3,
    points=2048,
    conditions=256,

    max_epochs=1000,
    batch_size=64,
)

In [7]:
encoder_hparams=dict(
    layers=[
        {"linear": 128},
        "gelu",
        {"linear": 256},
        {"multimax": 512},
        "gelu",
        {"linear": 512},
        "gelu",
        {"linear": 512},
        {"multimax": 256},
        "gelu",
        {"linear": 1024},
        "gelu",
        {"linear": 1024},
        {"multimax": 128},
        "gelu",
        {"linear": 1024},
        "gelu",
        {"linear": 1024},
        {"multimax": 64},
        "gelu",
        {"linear": 1024},
        "gelu",
        {"linear": 1024},
        {"multimax": 32},
        "gelu",
        {"linear": 1024},
        "gelu",
        {"linear": 2048},
        "gelu",
        "mean",
        {"linear": 2048},
        "gelu",
        {"linear": 1024},
        "gelu",
        {"linear": 512},
        "gelu",
        {"linear": hparams["conditions"]},
    ]
)

In [8]:
rectifier_hparams=dict(
    layer_widths=[128, 256, 512, 512, 256, 128],
    activations="gelu",
    integrator="euler"
)

In [9]:
hparams["encoder_hparams"] = encoder_hparams
hparams["rectifier_hparams"] = rectifier_hparams
hparams = PointCloudsModelHParams(**hparams)

In [10]:
train_dataset = FurnitureDataset(
    root="data/",
    shapes="all",
    split="train",
    sub_samples=hparams.points,
    samples=32768,
    download=False,
)

train_dataset, val_dataset = random_split(train_dataset, lengths=[0.8, 0.2])

In [11]:
test_dataset = FurnitureDataset(
    root="data/",
    shapes="all",
    split="test",
    sub_samples=hparams.points,
    samples=32768,
    download=False,
)

In [12]:
model = PointCloudsModel(hparams, train_data=train_dataset, val_data=val_dataset, test_data=test_dataset)

/home/lars/miniconda3/envs/rectifiers/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
model.fit()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      validation/mse       │     2.002831220626831     │
└───────────────────────────┴───────────────────────────┘

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EpochProgressBar, LearningRateMonitor, ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | encoder   | InvariantEncoder | 16.3 M
1 | rectifier | Rectifier        | 625 K 
-----------------------------------------------
16.9 M    Trainable params
0         Non-trainable params
16.9 M    Total params
67.703    Total estimated model params size (MB)
Epoch 6:   1%|          | 7/1000 [01:34<3:50:59, 13.96s/it, v_num=1]

In [ ]:
# checkpoint = "lightning_logs/version_1/checkpoints/last.ckpt"
# model = PointCloudsModel.load_from_checkpoint(checkpoint)

In [ ]:
samples = model.sample((4, 256)).cpu()

fig = viz.multiscatter(samples)
plt.show()